In [3]:
import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy import stats
import matplotlib.pyplot as plt
from tesser import util
from tesser import network
from tesser import rsa
plt.rcParams['figure.figsize'] = [10, 10]
import sys
np.set_printoptions(threshold=sys.maxsize)

ModuleNotFoundError: No module named 'tesser'

In [ ]:
#check RSA fies

item_rsa_file = '/Users/athulapudhiyidath/Dropbox/Experiments/tesser_successor/Data/item_betas/100_betas_10m.txt'
avg_rsa_file = 

In [2]:
###########
#DIRECTORY#
###########
#need to change path depending on whether you're using laptop or desktop:
#desktop:
#data_dir = '/Users/athula/Dropbox/Experiments/TesserScan/neural_results_2020/rsa_allevents'
#event_dir = '/Users/athula/Dropbox/Experiments/TesserScan/Neural_results_2020/rsa_allevents_info'

#laptop:
data_dir = '/Users/athulapudhiyidath/Dropbox/Experiments/tesser_successor/Data/item_betas'
event_dir = '/Users/athulapudhiyidath/Dropbox/Experiments/tesser_successor/TesserScan/rsa_allevents_info'

In [3]:
################
#SUBJECT + ROIs#
################

#load subject data:
subjs = util.subj_list()
subjs_np = np.array(subjs)

#load txt files by subj + ROI:
rois = ['b_hip_ant', 'b_hip_body', 'b_hip_tail', 'b_hip', 'l_hip_ant', 'l_hip_body', 'l_hip_tail', 'l_hip', 'r_hip_ant', 'r_hip_body', 'r_hip_tail', 'r_hip']
rois_np = np.repeat(rois,2)

#conditions
conditions = ('within', 'across')
conditions_np = np.array(conditions*len(rois))
conditions_array = [rois_np, conditions_np]

In [4]:
rsa_file = '/Users/athulapudhiyidath/Dropbox/Experiments/tesser_successor/Data/item_betas/100_betas_10m.txt'
rsa_df = np.loadtxt(rsa_file)

In [5]:
rsa_df.shape

(906, 906)

In [8]:
within_across_roi_overall_subj = np.array([])
within_across_roi_overall_mean = np.array([])
for r in range(len(rois)): 
    #loop through rois 
    roi = rois[r]
    
    within = []
    across = []
    for s in range(len(subjs)):
        #loop through subjects    
        sub = subjs[s]
        
        #getting RSA for a subject + their ROI
        this_auto_rsa = rsa.load_rsa(data_dir, sub, roi)

        #getting the community membership of each item across all runs 
        #getting the run numbers of each item
        #getting the scrambled numbers of each item
        run_all_items = []
        run_all_runs = []
        run_all_seq = []
        for run in list(range(1, 7)):
            this_run_info = util.load_struct_run_info(event_dir, sub, run)
            this_run_item = this_run_info.item
            run_all_items.extend(this_run_item)
    
            run_num = [run]*len(this_run_info)
            run_all_runs.extend(run_num)
    
            run_seq = this_run_info.seq_type
            run_all_seq.extend(run_seq)    

        run_all_comms = []
        for i in range(len(run_all_items)):
            item  = run_all_items[i]
            node_info = network.temp_node_info()
            if item != 0:
                item_comm = node_info.comm[item]
            else:
                item_comm = 'NaN'
        
            run_all_comms.append(item_comm)

        scram_all_seq = []
        nonscram_all_seq = []
        for i in range(len(run_all_seq)):
            seq  = run_all_seq[i]
            if seq == 1:
                scram = 0
                nonscram = 1
            elif seq == 2:
                scram = 1
                nonscram = 0 
            else:
                scram = 'NaN'
                nonscram = 'NaN'
            scram_all_seq.append(scram)
            nonscram_all_seq.append(nonscram)

        nan_all_seq = []
        for i in range(len(run_all_items)):
            obj  = run_all_items[i]
            if obj == 0:
                this_nan = 'NaN'
            else:
                this_nan = 0
            nan_all_seq.append(this_nan)
    
        run_all_comms = np.array(run_all_comms)
        run_all_runs = np.array(run_all_runs)
        run_all_scram = np.array(scram_all_seq)
        run_all_nonscram = np.array(nonscram_all_seq)
        run_all_nans = np.array(nan_all_seq)
        run_all_items = np.array(run_all_items)
        
        #assigning the 906 x 906 matrix with identifiers that indicate: 
        #within community = true 
        #across community = false
        within_comm = rsa.pair_eq(run_all_comms)
        #within_comm.shape
        #within_comm

        #assigning the 906 x 906 matrix with identifiers that indicate: 
        #within run = false 
        #across community = true
        across_run = rsa.pair_neq(run_all_runs)
        #across_run.shape
        #across_run

        #assigning the 906 x 906 matrix with identifiers that indicate: 
        #scram = true 
        #non-scram = false
        scram_run = rsa.pair_eq(run_all_scram)
        #scram_run.shape
        #scram_run

        #assigning the 906 x 906 matrix with identifiers that indicate: 
        #NaN item = 1 
        #other item = 0
        nan_run = np.zeros((len(run_all_nans), len(run_all_nans)))
        exclude = np.where(run_all_nans == 'NaN')
        nan_run[:, exclude] = 1
        nan_run[exclude, :] = 1

        #assigning the 906 x 906 matrix with identifiers that indicate: 
        #same object = true 
        #different object = false
        same_object = rsa.pair_eq(run_all_items)
        #same_object.shape
        #same_object  
        
        within_comm_sim = this_auto_rsa[(within_comm == True) & (across_run == True) & (scram_run == True) & (nan_run == 0) & (same_object == False)]
        across_comm_sim = this_auto_rsa[(within_comm == False) & (across_run == True) & (scram_run == True) & (nan_run == 0) & (same_object == False)]

        within_comm_sim_mean = np.nanmean(within_comm_sim)
        across_comm_sim_mean = np.nanmean(across_comm_sim)
        
        within.append(within_comm_sim_mean)
        across.append(across_comm_sim_mean)

    #by ROI
    within_roi_mean = np.mean(within)
    across_roi_mean = np.mean(across)
    
    within_across_roi_mean = np.column_stack([within_roi_mean, across_roi_mean])
    within_across_roi_all = np.column_stack([within, across])
    
    if r == 0:
        within_across_roi_overall_mean =  within_across_roi_mean
        within_across_roi_mean_tog_all = within_across_roi_all
    else:
        within_across_roi_overall_mean = np.concatenate([within_across_roi_overall_mean, within_across_roi_mean])
        within_across_roi_mean_tog_all = np.concatenate([within_across_roi_mean_tog_all, within_across_roi_all], axis=1)

In [10]:
within_across_roi_overall_mean

array([[0.00096264, 0.0009311 ],
       [0.0022928 , 0.00220058],
       [0.00267829, 0.00249241],
       [0.00264978, 0.00250664],
       [0.00076157, 0.00074646],
       [0.0021633 , 0.00186986],
       [0.00228445, 0.00230842],
       [0.00216287, 0.00199856],
       [0.00132347, 0.00124796],
       [0.00248995, 0.00243105],
       [0.00284956, 0.00261424],
       [0.00310909, 0.00299515]])

In [11]:
#MEAN ROI + WITHIN/ACROSS 
roi_mean_sim_df = pd.DataFrame(within_across_roi_overall_mean,
                  index=[rois],
                  columns=['within', 'across'])

roi_mean_sim_df

,within,across
b_hip_ant,0.000963,0.000931
b_hip_body,0.002293,0.002201
b_hip_tail,0.002678,0.002492
b_hip,0.002650,0.002507
l_hip_ant,0.000762,0.000746
l_hip_body,0.002163,0.001870
l_hip_tail,0.002284,0.002308
l_hip,0.002163,0.001999
r_hip_ant,0.001323,0.001248
r_hip_body,0.002490,0.002431


In [12]:
roi_mean_sim_df.to_csv('singletrial_scrambled_roi_WithinAcross.csv', index=True)  

In [15]:
within_across_roi_mean_tog_all

array([[ 2.05183320e-03,  2.66300733e-03, -3.89926864e-04,
         1.22205498e-04,  1.76943340e-03,  3.86648863e-03,
         1.15921664e-03,  2.04905622e-03,  2.11836907e-03,
         2.67603943e-03,  3.91674010e-04,  1.14237375e-03,
         3.43745973e-03,  6.24233399e-03,  1.82536687e-03,
         2.95898297e-03,  2.39616481e-03,  3.06840241e-03,
        -6.23416136e-04, -7.76031866e-04, -7.59013790e-04,
         5.48380222e-04,  6.66449314e-04,  1.12438143e-03],
       [ 1.13733116e-03,  1.38397646e-03,  2.26168479e-03,
         1.96032495e-03,  2.89362433e-03,  2.05840187e-03,
         2.36443718e-03,  1.65749687e-03,  2.08416203e-03,
         2.79947214e-03,  2.94960536e-03,  1.09490236e-03,
         1.46127876e-03,  1.14137805e-03,  2.87162002e-03,
         2.40733929e-03,  7.66243501e-04,  3.29445818e-04,
         2.21090993e-03,  2.90556946e-03,  1.46984802e-03,
        -5.99716190e-04,  1.72405001e-03,  9.59142101e-04],
       [ 8.84238296e-05,  3.72928151e-04,  2.76401119e

In [13]:
#MEAN ROI + WITHIN/ACROSS x SUBJECT:
roi_mean_subj_sim_df = pd.DataFrame(within_across_roi_mean_tog_all, index=[subjs_np], columns=conditions_array)
roi_mean_subj_sim_df

b_hip_ant           b_hip_body           b_hip_tail            \
           within    across     within    across     within    across   
100  2.051833e-03  0.002663  -0.000390  0.000122   0.001769  0.003866   
101  1.137331e-03  0.001384   0.002262  0.001960   0.002894  0.002058   
102  8.842383e-05  0.000373   0.000276  0.000661  -0.000273  0.000692   
103  7.187308e-04  0.001260   0.002695  0.002093  -0.000668 -0.000350   
104  7.438463e-04  0.000481   0.001118  0.001527   0.001122  0.002018   
105  1.006098e-03  0.001308   0.001368  0.001110   0.004449  0.003518   
106  1.844807e-03  0.000670   0.002012  0.001917   0.002883  0.003120   
107 -1.726828e-03 -0.001779   0.000518 -0.000221   0.003187  0.003350   
108  6.472884e-04 -0.000903   0.004770  0.000398   0.004538  0.002390   
109 -9.102142e-04 -0.001309   0.001146  0.000195   0.001384  0.000684   
110  1.694918e-03  0.000495   0.003019  0.002168   0.006617  0.004920   
111  2.129002e-03  0.002704   0.004377  0.005771  -0.000931 -0.001158   
112 -3.548023e-05 -0.000061   0.000630  0.000524   0.000259  0.000464   
113  1.048410e-03  0.000359   0.007434  0.006574   0.002467  0.000874   
114  1.370876e-03  0.000475   0.003906  0.004599   0.003035  0.003721   
115  8.451716e-04  0.000497   0.000600  0.001716   0.002445  0.002373   
116 -4.061669e-04 -0.000522   0.009526  0.010061   0.009753  0.011423   
117  4.607491e-04  0.002009   0.004727  0.005165   0.009953  0.010267   
119  1.415725e-04  0.000221   0.001764  0.002039   0.003806  0.002494   
120  5.008028e-04  0.000147   0.000461  0.000947   0.002601  0.001521   
121 -4.072278e-04  0.000212  -0.000949 -0.000524   0.002634  0.001356   
122 -8.908048e-07  0.000608  -0.000599  0.000289   0.000926  0.000569   
123 -1.569028e-04 -0.000119  -0.000184 -0.001361   0.001604 -0.000152   
124  1.769969e-03  0.002251   0.004534  0.005566   0.002414  0.002423   
125 -7.562239e-05  0.000288   0.001284  0.002292  -0.000165  0.000238   
126  2.888764e-03  0.002941   0.000051  0.000224   0.001185  0.000987   
127  1.743156e-04 -0.000964   0.000011 -0.000488   0.000848  0.000350   
128 -9.522500e-04 -0.000256  -0.000440 -0.000235   0.000784  0.000609   
129  2.436796e-03  0.001937   0.005556  0.004811   0.002104  0.001408   
130  1.272236e-03  0.001133   0.002612  0.003701   0.002930  0.002286   
131  8.573740e-03  0.009481   0.011456  0.011289   0.006826  0.007578   
132  6.983094e-04  0.000681  -0.001713 -0.001153   0.001452  0.002508   
133  1.181978e-03  0.001043   0.002559  0.001467   0.006939  0.005446   
135  2.586143e-03  0.002018   0.002301  0.000102   0.004737  0.005745   
136 -1.412175e-04  0.000351   0.003524  0.003060   0.001236  0.000522   
137  1.836240e-03  0.001871   0.002332  0.001822   0.000174  0.001523   
138  5.820000e-04  0.000502   0.000279  0.001232   0.001179  0.000575   

        b_hip           l_hip_ant            ...     l_hip            \
       within    across    within    across  ...    within    across   
100  0.001159  0.002049  0.002118  0.002676  ...  0.001825  0.002959   
101  0.002364  0.001657  0.002084  0.002799  ...  0.002872  0.002407   
102  0.000214  0.000837  0.000098  0.000057  ...  0.000031  0.000673   
103  0.001459  0.001365  0.000871  0.001924  ...  0.001283  0.001315   
104  0.001392  0.001536  0.000106 -0.000423  ...  0.000354  0.000270   
105  0.003098  0.003131  0.002046  0.001803  ...  0.002726  0.002279   
106  0.002829  0.002091  0.003303  0.002761  ...  0.003868  0.002803   
107  0.000867 -0.000089 -0.000919 -0.001146  ...  0.000987  0.000058   
108  0.003260  0.000398 -0.000199 -0.000740  ...  0.003985  0.000462   
109  0.000671  0.000403 -0.002682 -0.002198  ...  0.000804  0.000540   
110  0.003424  0.002290  0.002330  0.000942  ...  0.001393  0.000116   
111  0.001857  0.002467  0.001225  0.001204  ...  0.001923  0.002521   
112  0.001255  0.001171  0.000217  0.000472  ...  0.000666  0.000652   
113  0.003951  0.003190  0.001367 -0.000002  ...  0.005506  0.004107   
114  0.0

In [14]:
roi_mean_subj_sim_df.to_csv('singletrial_scrambled_roi_WithinAcross_Subject.csv', index=True)  